# Turkish NLP Pipeline with BERT 

1. Sentiment Analysis
2. NER Model
3. Question Answering
4. Text Summarization
5. Text Categorization

These models are fined tuned based on Turkish-Bert model 

https://github.com/stefan-it/turkish-bert

## Requirements
* python3
* pip install transformers

In [2]:
#!pip install transformers

In [4]:
import transformers

## 1-Sentiment Analysis
* This model can get up to %95 success rate on our dataset 
* To see the training detail and the model performce, check the link \
 https://huggingface.co/savasy/bert-base-turkish-sentiment-cased
 
 https://github.com/savasy/TurkishSentimentAnalysis
 
 

### Usage

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
# load model, it takes time since it loads over 500 MB model file
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

# create pipeline
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

### Example usage
* Label_1: positive
* Label_0: negative

In [10]:
p= sa("bu telefon modelleri çok kaliteli ve ayrıca ucuz ve  kolay bulunuyor")
print(p)
#[{'label': 'LABEL_1', 'score': 0.9871089}]
print (p[0]['label']=='LABEL_1')
#True


[{'label': 'LABEL_1', 'score': 0.5605083107948303}]
True


In [11]:
p= sa("Film çok kötü ve oyuncular çok sahteydi")
print(p)
#[{'label': 'LABEL_0', 'score': 0.9975505}]
print (p[0]['label']=='LABEL_1')
#False

[{'label': 'LABEL_0', 'score': 0.9989525079727173}]
False


### Test your entire dataset

> Suppose your file has lots of lines of comment and label (1 or 0) (tab seperated)

```
 # yourfile.tsv
 comment1 ... \t label
 comment2 ... \t label

 comment-n ... \t  label
...
```



# your test code

```
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

f="/path/to/your/file/yourfile.tsv"

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

i,crr=0,0
for line in open(f):
 lines=line.strip().split("\t")
 if len(lines)==2:
  i=i+1
  if i%100==0:
   print(i)
  pred= sa(lines[0])
  pred=pred[0]["label"].split("_")[1]
  if pred== lines[1]:
   crr=crr+1

print(crr, i, crr/i)
```

How to train and fine-tune your own model, please check 

https://github.com/savasy/TurkishSentimentAnalysis

# 2- Name Entity Recognizer (NER)
This model can get %95 accuracy, currently it works with PER, LOC, and ORG

check huugingface model repo and ner pipeline repo for other detail

* https://huggingface.co/savasy/bert-base-turkish-ner-cased
* https://github.com/savasy/Turkish-Bert-Based-NERModel


## Example usage

In [16]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")

ner=pipeline('ner', model=model, tokenizer=tokenizer)
res=ner("Mustafa Kemal Atatürk 19 Mayıs 1919'da Samsun'a ayak bastı.")
for r in res:
    print(r)


{'word': 'Mustafa', 'score': 0.9938516616821289, 'entity': 'B-PER', 'index': 1}
{'word': 'Kemal', 'score': 0.9881671071052551, 'entity': 'I-PER', 'index': 2}
{'word': 'Atatürk', 'score': 0.9957979321479797, 'entity': 'I-PER', 'index': 3}
{'word': 'Samsun', 'score': 0.9059973359107971, 'entity': 'B-LOC', 'index': 9}


In [17]:
res=ner("Sait Faik ömrünün sonuna kadar yazları Burgaz adadaki köşklerinde kalmıştır")

for r in res:
    print(r)


{'word': 'Sait', 'score': 0.9995592832565308, 'entity': 'B-PER', 'index': 1}
{'word': 'Faik', 'score': 0.999716579914093, 'entity': 'I-PER', 'index': 2}
{'word': 'Bur', 'score': 0.7886330485343933, 'entity': 'B-LOC', 'index': 9}
{'word': '##gaz', 'score': 0.8438959121704102, 'entity': 'I-LOC', 'index': 10}


# 3- Question Answering  (SQuAD) for Turkish

This model is tranied with TQuAD dataset (which is SQuAD-like data set of Turkish)

https://github.com/TQuad/turkish-nlp-qa-dataset

> This dataset is the Turkish Question & Answer dataset on Turkish & Islamic Science History within the scope of Teknofest 2018 Artificial Intelligence competition.


# Loading model

In [19]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-squad")
model = AutoModelForQuestionAnswering.from_pretrained("savasy/bert-base-turkish-squad")

nlp=pipeline("question-answering", model=model, tokenizer=tokenizer)

In [22]:
sait="ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."

In [23]:
print(nlp(question="Ne zaman avare bir hayata başladı?", context=sait))

{'score': 0.9998962879427609, 'start': 752, 'end': 775, 'answer': 'Babasının ölümü üzerine'}


In [24]:
print(nlp(question="Sait Faik hangi Lisede orta öğrenimini tamamladı?", context=sait))

{'score': 0.9957321713650771, 'start': 246, 'end': 262, 'answer': 'Bursa Lisesi’nde'}


In [25]:
print(nlp(question="Sait Faik bir film yönetmeni mi?", context=sait))

{'score': 0.0057270539825189926, 'start': 75, 'end': 81, 'answer': '1954).'}


In [26]:
# Ask your self ! type your question
print(nlp(question="...?", context=sait))

{'score': 0.008398349746329359, 'start': 75, 'end': 81, 'answer': '1954).'}


In [27]:
# source: wikipedia
ataturk="Atatürk  modern, ilerici ve laik bir ulus devleti oluşturmak için politik, ekonomik ve kültürel alanlarda sekülarist ve milliyetçi \
 karakterdeki reformlarını başlattı. Yabancılara tanınan ekonomik imtiyazlar kaldırıldı ve onlara ait üretim araçları ve demiryolları millîleştirildi. \
 Tevhîd-i Tedrîsât Kanunu ile eğitim Türk hükûmetinin denetimine girdi. Seküler ve bilimsel eğitim esas alındı. Binlerce yeni okul inşa edildi. \
 İlköğretim ücretsiz ve zorunlu hale getirildi. Yabancı okullar devlet denetimine alındı. Köylülerin sırtına yüklenen ağır vergiler azaltıldı. \
 Erkeklerin serpuş ve kıyafetlerinde değişiklikler yapıldı. Takvim, saat ve ölçülerde değişikliklere gidildi. \
 Mecelle kaldırılarak yerine seküler Türk Kanunu Medenisi yürürlüğe konuldu. Kadınların sivil ve politik hakları pek çok Batı ülkesinden önce tanındı. \
 Çok eşlilik yasaklandı. Kadınların şahitliği ve miras hakkı erkeklerinkiyle eşit hale getirildi. Benzer şekilde, dünyanın çoğu ülkesinden önce olarak \
 Türkiye'de kadınların ilkin yerel seçimlerde (1930), sonra genel seçimlerde (1934) seçme ve seçilme hakkı tanındı. Ceza ve borçlar hukukunda \
 seküler yasalar yürürlüğe konuldu. Sanayi Teşvik Kanunu kabul edildi. Toprak Reformu için çabalandı.[3] Arap harfleri temelli Osmanlı alfabesinin yerine \
  Latin harfleri temelli yeni Türk alfabesi kabul edildi. Halkı okuryazar kılmak için eğitim seferberliği başlatıldı. Üniversite Reformu gerçekleştirildi. \
  Birinci Beş Yıllık Sanayi Planı yürürlüğe konuldu. Sınıf ve statü farkı gözeten lâkap ve unvanlar kaldırıldı ve soyadları yürürlüğe konuldu. \
  Homojen ve birleşmiş bir ulus yaratılması için Türkleştirme politikası yürütüldü.[4][5][6] Türk olmayan azınlıklar kamuoyunda Türkçe konuşmaya zorlandı,[7] \
  Türkçe olmayan toponomiler ve azınlıkların soyadları Türkçeye çevrildi.[8][9]"

In [28]:
print(nlp(question="Kimlere ekonomik imtiyaz kaldırıldı?", context=ataturk))

{'score': 0.9999153630840389, 'start': 168, 'end': 179, 'answer': 'Yabancılara'}


In [29]:
print(nlp(question="tüm bu devrimleri kim yaptı?", context=ataturk))

{'score': 1.3106793647542953e-09, 'start': 0, 'end': 8, 'answer': 'Atatürk'}


In [30]:
# Ask your self ! type your question
print(nlp(question="...?", context=ataturk))

{'score': 9.457354802119247e-13, 'start': 1434, 'end': 1484, 'answer': 'Birinci Beş Yıllık Sanayi Planı yürürlüğe konuldu.'}


# 4 - Text Summarization

...will be soon

# 5- Text/Document Classification 

In this study we fine-tune 7 classes Turkish news dataset


The following Turkish benchmark dataset is used for fine-tuning

https://www.kaggle.com/savasy/ttc4900

 * 'LABEL_0': 'dunya 
 * 'LABEL_1': 'ekonomi '
 * 'LABEL_2': 'kultur '
 * 'LABEL_3': 'saglik '
 * 'LABEL_4': 'siyaset '
 * 'LABEL_5': 'spor '
 * 'LABEL_6': 'teknoloji'

In [35]:
# import libraries
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification

# load models
tokenizer= AutoTokenizer.from_pretrained("savasy/bert-turkish-text-classification")
model= AutoModelForSequenceClassification.from_pretrained("savasy/bert-turkish-text-classification")


In [36]:
# make pipeline
nlp=pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# apply model
res=nlp("bla bla")
res
# [{'label': 'LABEL_2', 'score': 0.4753005802631378}]

[{'label': 'LABEL_2', 'score': 0.4753005802631378}]

In [37]:
code_to_label={
 'LABEL_0': 'dunya ',
 'LABEL_1': 'ekonomi ',
 'LABEL_2': 'kultur ',
 'LABEL_3': 'saglik ',
 'LABEL_4': 'siyaset ',
 'LABEL_5': 'spor ',
 'LABEL_6': 'teknoloji '}
# > 'kultur '


In [38]:
code_to_label[nlp("bla bla")[0]['label']]

'kultur '

In [39]:
text="Bugün 19 Mayıs Atatürk'ü Anma, Gençlik ve Spor Bayramı. Mustafa Kemal Atatürk, 19 Mayıs 1919'da Bandırma Vapuru ile Samsun'a çıkmış ve İtilaf Devletleri'nin işgaline karşı Kurtuluş Savaşı'nı başlatmıştı. Siyaset Bilimci Prof. Dr. Ahmet Demirel, Mustafa Kemal'in askeri başarılarıyla tanınan bir Osmanlı paşasıyken tüm yurt çapında saygı duyulan siyasi bir lidere dönüşmesinin ilk adımı olan 19 Mayıs 1919'un öncesi ve sonrasında neler yaşandığını BBC Türkçe'ye anlattı."
code_to_label[nlp(text)[0]['label']]

'siyaset '

For the details of traininig models please check

* https://github.com/savasy/TurkishTextClassification